In [26]:
import pandas as pd
from torchsummary import summary
import random
import sys 
import os
import numpy as np
import torch
from torch.utils.data import DataLoader, random_split
import torch.optim as optim

In [27]:
# Get the project directory 
current_dir = os.path.abspath('') # Current '\notebooks' directory
project_dir = os.path.abspath(os.path.join(current_dir, '..')) # Move up one level to project root directory

# Add the project directory to sys.path
sys.path.append(project_dir)

# Move up to project directory
os.chdir(project_dir)
os.getcwd()

'/Users/aishwaryaiyer/Documents'

In [28]:
from src.models import *
from src.dataset import *



In [29]:
transformer_model_path = "/Users/aishwaryaiyer/Documents/GitHub/Digital-Asset-Prediction/saved_models/CryptoTransformer_2025-04-09_21-31-23/CryptoTransformer_BEST_R2.pth"
informer_model_path = "/Users/aishwaryaiyer/Documents/GitHub/Digital-Asset-Prediction/saved_models/CryptoInformer_2025-04-10_15-11-19/CryptoInformer_BEST_R2.pth"

In [30]:
transformer_model = CryptoTransformer()  
informer_model = CryptoInformer()


In [31]:
transformer_model.load_state_dict(torch.load(transformer_model_path, map_location=torch.device('cpu')))
informer_model.load_state_dict(torch.load(informer_model_path, map_location=torch.device('cpu')))

transformer_model.eval()
informer_model.eval()

/var/folders/vr/qwxtvrr17fn2qzlm0z4rq4br0000gn/T/ipykernel_72818/2746541776.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  transformer_model.load_state_dict(torch.load(

CryptoInformer(
  (embedding): Linear(in_features=11, out_features=64, bias=True)
  (positional_encoder): SinusoidalPositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (attn): ProbAttention(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder_layers): ModuleList(
    (0-1): 2 x EncoderLayer(
      (attention): AttentionLayer(
        (inner_attention): ProbAttention(
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (query_projection): Linear(in_features=64, out_features=64, bias=True)
        (key_projection): Linear(in_features=64, out_features=64, bias=True)
        (value_projection): Linear(in_features=64, out_features=64, bias=True)
        (out_projection): Linear(in_features=64, out_features=64, bias=True)
      )
      (conv1): Conv1d(64, 256, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(256, 64, kernel_size=(1,), stride=(1,))
      (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((64,)

In [32]:
class CryptoDataset(Dataset):
    def __init__(self, data: pd.DataFrame, features: list, target: str = None, sequence_length: int = 14):
        self.data = data[features].values
        self.target = data[target].values if target else None
        self.sequence_length = sequence_length
        
    def __len__(self):
        # Return 1 since we just want the full sequence for prediction
        return 1 if len(self.data) == self.sequence_length else len(self.data) - self.sequence_length
        
    def __getitem__(self, idx) -> Tuple[torch.Tensor, torch.Tensor]:
        seq = self.data[idx:idx+self.sequence_length]
        if self.target is not None and (idx + self.sequence_length) < len(self.data):
            target = self.target[idx+self.sequence_length]
            return torch.FloatTensor(seq), torch.FloatTensor([target])
        return torch.FloatTensor(seq), torch.FloatTensor([0])  # Dummy target for prediction

In [33]:
features = ['open', 'high', 'low', 'close', 'volume', 'market_cap', 
            'daily_return', 'sp500', 'treasury_spread', 'fear_greed', 'gold_price_usd']
target = 'close'

# Create a single sequence dataset
dataset = CryptoDataset(ticker_data, features, target)
seq, _ = dataset[0]  # Get our 14-day sequence (shape: [14, num_features])


In [34]:
def predict_ticker(ticker_data, model, model_name):
    dataset = CryptoDataset(ticker_data, features, target=None)
    seq, _ = dataset[0]
    
    with torch.no_grad():
        input_seq = seq.unsqueeze(0)
        normalizer = Normalizer()
        normalizer.fit(input_seq)
        pred = model(input_seq)
        
    return pred.item()

In [35]:
all_tickers = df['symbol'].unique()
selected_tickers = np.random.choice(all_tickers, size=20, replace=False)

# Prepare results DataFrame
results = pd.DataFrame(columns=[
    'Ticker', 
    'Last_Close', 
    'Transformer_Pred', 
    'Informer_Pred',
    'Transformer_Diff',
    'Informer_Diff'
])

# Get predictions for each ticker
for ticker in selected_tickers:
    ticker_data = df[df['symbol'] == ticker].tail(14)
    
    if len(ticker_data) < 14:
        continue  # Skip if not enough data
        
    last_close = ticker_data['close'].iloc[-1]
    
    transformer_pred = predict_ticker(ticker_data, transformer_model, "Transformer")
    informer_pred = predict_ticker(ticker_data, informer_model, "Informer")
    
    # Calculate absolute differences
    transformer_diff = abs(transformer_pred - last_close)
    informer_diff = abs(informer_pred - last_close)
    
    # Add to results
    results.loc[len(results)] = {
        'Ticker': ticker,
        'Last_Close': last_close,
        'Transformer_Pred': transformer_pred,
        'Informer_Pred': informer_pred,
        'Transformer_Diff': transformer_diff,
        'Informer_Diff': informer_diff
    }

# Calculate some statistics
results['Transformer_Error_Pct'] = (results['Transformer_Diff'] / results['Last_Close']) * 100
results['Informer_Error_Pct'] = (results['Informer_Diff'] / results['Last_Close']) * 100

# Save to CSV
output_path = "predictions.csv"
results.to_csv(output_path, index=False)

print(f"\nPredictions saved to: {output_path}")
print("\nSummary Statistics:")
print(results[['Transformer_Error_Pct', 'Informer_Error_Pct']].describe())

# Display first few results
print("\nSample Predictions:")
print(results.head())


Predictions saved to: predictions.csv

Summary Statistics:
       Transformer_Error_Pct  Informer_Error_Pct
count           2.000000e+01        2.000000e+01
mean            1.867799e+05        5.945227e+05
std             3.892810e+05        1.223386e+06
min             8.485883e+01        2.748114e+01
25%             1.662873e+03        6.092868e+03
50%             5.174402e+03        1.754093e+04
75%             1.162963e+05        3.780090e+05
max             1.481523e+06        4.571896e+06

Sample Predictions:
      Ticker  Last_Close  Transformer_Pred  Informer_Pred  Transformer_Diff  \
0   STX/USDT    0.685000        -10.865082      42.866081         11.550082   
1  DOGE/USDT    0.183000        -11.994514      40.923820         12.177514   
2  TROY/USDT    0.001277        -12.171100      39.731094         12.172377   
3   FUN/USDT    0.002370        -12.172416      39.173653         12.174786   
4  DUSK/USDT    0.096300        -12.075022      39.541847         12.171322   

   